# Working with a Directory of .CSV files
- I want to manipulate all the files in a similar way
- I want to add the filename of each file I am cleaning as a column so I can find which file the data came from later.

In [ ]:
# import some modules
import os
import os.path
import pandas as pd
import sqlite3
import numpy as np
import glob
import csv

## List out the files in the directory.

In [ ]:
mypath=r"C:\Users\CHUGHES\Documents\PROJECTS\Python\001-NCC\DATA\2017\CSV"
myfilelist = os.listdir(mypath)
for f in myfilelist:
    print(f)

## There is an issue with getting a trailing slash on the path

In [ ]:
dbconn = sqlite3.connect('ncc.db')
print(dbconn)

In [ ]:
cursor = dbconn.cursor()
cursor

In [ ]:
mypath

In [ ]:
mypath = os.path.join(mypath, '') # adds the needed slash on the end so the below will work
mypath

### Below states the headers I want for each column

In [ ]:
header = {0:"OFFICE",1:"LNAME",2:"FNAME",3:"MNAME",4:"ONAME",46:"NOTES"}
header

## Below makes a variable with all the file names to iterate through

In [ ]:
all_files = glob.glob(mypath + "*.csv")

In [ ]:
header

### Below reads in each file, removes blank lines in it and writes it back to the os. Should only have to run this block once.

In [ ]:
for filename in all_files:
    df = pd.read_csv(filename)
    df.dropna(how="all", inplace=True)
    df.rename(columns = header, inplace=True)
    df.to_csv(filename, index=False)

## Below reads in each file, replaces the header, Adds the filename as a column.

In [ ]:
df = pd.concat((pd.read_csv(f,
                                   skiprows=1,
                                   header=None,
                                   index_col=None,
                                   skip_blank_lines=True,
                                   usecols = [0,1,2,3,4,46],
                        ).assign(FILENAME=os.path.basename(f))
                            for f in glob.glob(os.path.join(mypath, '*.csv'))
                       ),
                       ignore_index=True)
df.rename(columns = header, inplace=True)

## IMPORT the dataframe into the database

In [ ]:
df.to_sql('TEST',con = dbconn, if_exists='append', index=False)

# BELOW HERE ARE COMMANDS TO SET UP / MAINTAIN THE DB

In [ ]:
dropTableStatement = "DROP TABLE TEST"

cursor.execute(dropTableStatement)

dbconn.commit()